In [3]:
import cppimport
import sys
sys.path.append('cppcode')

from config import opt

from models import Models
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from data.dataset import Data
import util
from datetime import datetime
from evaluate import Evaluator
from collections import defaultdict

backbone = MF, method = TACR-new, dataset = Amazon-Music, val = True
lr = 0.0010, lamb = 0.010000, batch_size = 8192, topk = 20
tau = 10000000, q = -1.00, b = -2.00
 lr_q = 0.001000, lr_b = 0.000100
high_popularity_quantile = 0.8, smooth_parameter = 256,
emb_dim = 64, smooth_parm_alpha = 0.5, dt_u = 10, dt_i = 5, rolling_window = 10
model_path: model/Amazon-Music/2024-12-17 21.09.04_MF_TACR-new.pth


In [4]:
# reload config, so the changed part will be effect
import importlib
import config
import models
importlib.reload(config)
importlib.reload(models)
from config import opt
from models import Models


backbone = MF, method = TACR-new, dataset = Amazon-Music, val = True
lr = 0.0010, lamb = 0.010000, batch_size = 8192, topk = 20
tau = 10000000, q = -1.00, b = -2.00
 lr_q = 0.001000, lr_b = 0.000100
high_popularity_quantile = 0.8, smooth_parameter = 256,
emb_dim = 64, smooth_parm_alpha = 0.5, dt_u = 10, dt_i = 5, rolling_window = 10
model_path: model/Amazon-Music/2024-12-17 21.09.34_MF_TACR-new.pth


In [5]:
def min_max_normalization(data):
    min_val = np.min(data)
    max_val = np.max(data)
    normalized_data = (data - min_val) / (max_val - min_val)
    return normalized_data

def z_score_normalization(data):
    mean_val = np.mean(data)
    std_val = np.std(data)
    normalized_data = (data - mean_val) / std_val
    return normalized_data

def max_abs_normalization(data):
    max_abs_val = np.max(np.abs(data))
    normalized_data = data / max_abs_val
    return normalized_data

def robust_scaler(data):
    median_val = np.median(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    normalized_data = (data - median_val) / iqr
    return normalized_data

In [6]:
dataset_name = opt.dataset # Amazon-Music Ciao Douban-movie

print(dataset_name)
main_path = 'data/'
train_data_path = main_path + '{}/train_data.csv'.format(dataset_name)
val_data_path = main_path + '{}/val_data.csv'.format(dataset_name)
test_data_path = main_path + '{}/test_data.csv'.format(dataset_name)

Amazon-Music


In [7]:
train_data = pd.read_csv(train_data_path, sep='\t')
val_data = pd.read_csv(val_data_path, sep='\t')
test_data = pd.read_csv(test_data_path, sep='\t')
all_data = pd.concat([train_data, val_data, test_data], ignore_index=True)
user_num = max(
    train_data['user'].max(),
    test_data['user'].max(),
    val_data['user'].max()) + 1
item_num = max(
    train_data['item'].max(),
    test_data['item'].max(),
    val_data['item'].max()) + 1
min_time = train_data['timestamp'].min()
max_time = train_data['timestamp'].max()

user_num, item_num, min_time, max_time

(5541, 3568, 893721600, 1354752000)

In [9]:
test_set = defaultdict(list)

test_user = test_data.user.unique()
for i in range(test_data.shape[0]):
    uid = test_data.loc[i, 'user']
    item_id = test_data.loc[i, 'item']
    test_set[uid].append(item_id)

### short-term user popularity sensitivity drawing

In [18]:
# reload config, so the changed part will be effect
import importlib
import config
importlib.reload(config)
from config import opt

backbone = MF, method = TACR-new, dataset = Amazon-Music, val = True
lr = 0.0010, lamb = 0.010000, batch_size = 8192, topk = 20
tau = 10000000, q = -1.00, b = -2.00
 lr_q = 0.001000, lr_b = 0.000100
high_popularity_quantile = 0.8, smooth_parameter = 256,
emb_dim = 64, smooth_parm_alpha = 0.5, dt_u = 10, dt_i = 5, rolling_window = 10
model_path: model/Amazon-Music/2024-12-17 21.11.06_MF_TACR-new.pth


In [148]:
dataset_name = opt.dataset # Amazon-Music Ciao Douban-movie

print(dataset_name)
main_path = 'data/'
train_data_path = main_path + '{}/train_data.csv'.format(dataset_name)
val_data_path = main_path + '{}/val_data.csv'.format(dataset_name)
test_data_path = main_path + '{}/test_data.csv'.format(dataset_name)

Amazon-Music


In [149]:
train_data = pd.read_csv(train_data_path, sep='\t')
val_data = pd.read_csv(val_data_path, sep='\t')
test_data = pd.read_csv(test_data_path, sep='\t')
all_data = pd.concat([train_data, val_data, test_data], ignore_index=True)
user_num = max(
    train_data['user'].max(),
    test_data['user'].max(),
    val_data['user'].max()) + 1
item_num = max(
    train_data['item'].max(),
    test_data['item'].max(),
    val_data['item'].max()) + 1
# print(user_num, item_num, len(train_data) + len(val_data) + len(test_data))
min_time = train_data['timestamp'].min()
max_time = train_data['timestamp'].max()

user_num, item_num, min_time, max_time, \
datetime.fromtimestamp(min_time), datetime.fromtimestamp(max_time), \
datetime.fromtimestamp(all_data['timestamp'].min()), \
datetime.fromtimestamp(all_data['timestamp'].max())

(5541,
 3568,
 893721600,
 1354752000,
 datetime.datetime(1998, 4, 28, 9, 0),
 datetime.datetime(2012, 12, 6, 9, 0),
 datetime.datetime(1998, 4, 28, 9, 0),
 datetime.datetime(2014, 7, 23, 9, 0))

In [ ]:
# train data distribution

# 统计各个用户的出现频率
user_frequency = train_data['user'].value_counts()

# 统计各个物品的出现频率
item_frequency = train_data['item'].value_counts()

items = item_frequency.sort_values().index
# item_frequency.sort_values().values
all_frequency = item_frequency.unique()

items

In [ ]:
half_year_step_ts = 365//2 * 24 * 60 * 60

ts_step = (train_data['timestamp'].max() - train_data['timestamp'].min() - half_year_step_ts)//100

all_times = train_data['timestamp'].min() + half_year_step_ts + np.arange(0, 101) * ts_step


In [ ]:
# short-term popularity

quantile = 0.8
item_frequency_all_times = {}
user_frequency_all_times = {}
user_frequency_high_popularity_all_times = {}
for time_idx in range(len(all_times)):
    if time_idx == len(all_times)-1:
        cur_ts = all_times[time_idx] - half_year_step_ts
        next_ts = train_data['timestamp'].max()
    elif time_idx==0:
        cur_ts = train_data['timestamp'].min()
        next_ts = all_times[time_idx]
    else:
        cur_ts = all_times[time_idx] - half_year_step_ts
        next_ts = all_times[time_idx]
    data_between_ts = train_data[np.logical_and(train_data['timestamp'] >= cur_ts, train_data['timestamp'] <= next_ts)]
    item_frequency_between_ts = data_between_ts['item'].value_counts()
    item_frequency_all_times[time_idx] = item_frequency_between_ts
    
    user_frequency_between_ts = data_between_ts['user'].value_counts()
    user_frequency_all_times[time_idx] = user_frequency_between_ts
    
    # get the threshold of high_frequency_items
    item_high_popularity_threshold = item_frequency_between_ts.quantile(quantile)
    high_popularity_items = (item_frequency_between_ts[item_frequency_between_ts>item_high_popularity_threshold]).index
    # get the frequency of user's clicks on high_popularity items
    data_between_ts_high_popularity = data_between_ts[data_between_ts['item'].isin(high_popularity_items)]
    user_frequency_high_popularity_between_ts = data_between_ts_high_popularity['user'].value_counts()
    user_frequency_high_popularity_all_times[time_idx] = user_frequency_high_popularity_between_ts
    
item_frequency_all_times = pd.DataFrame(item_frequency_all_times)
user_frequency_all_times = pd.DataFrame(user_frequency_all_times)
user_frequency_high_popularity_all_times = pd.DataFrame(user_frequency_high_popularity_all_times)


item_frequency_all_times = item_frequency_all_times.reindex(item_frequency.index).sort_index()
user_frequency_all_times = user_frequency_all_times.reindex(user_frequency.index).sort_index()
user_frequency_high_popularity_all_times = user_frequency_high_popularity_all_times.reindex(user_frequency.index).sort_index()


user_frequency_high_popularity_all_times.fillna(0)/(user_frequency_all_times.fillna(0)+1)

In [152]:
# # save short-term item frequency, user frequency, user frequency on high popularity items
save_file_name_item_freq = "data/{}/item_frequency_all_times.csv".format(dataset_name)
save_file_name_user_freq = "data/{}/user_frequency_all_times.csv".format(dataset_name)
save_file_name_user_freq_high_popu = "data/{}/user_frequency_high_popularity_all_times.csv".format(dataset_name)
save_file_all_times = "data/{}/all_times.npy".format(dataset_name)


# print(save_file_name_item_freq)
# print(save_file_name_user_freq)
# print(save_file_name_user_freq_high_popu)

item_frequency_all_times.to_csv(save_file_name_item_freq)
user_frequency_all_times.to_csv(save_file_name_user_freq)
user_frequency_high_popularity_all_times.to_csv(save_file_name_user_freq_high_popu)
np.save(save_file_all_times, all_times)

# item_frequency_all_times = pd.read_csv(save_file_name_item_freq, index_col=0)
# user_frequency_all_times = pd.read_csv(save_file_name_user_freq, index_col=0)
# user_frequency_high_popularity_all_times = pd.read_csv(save_file_name_user_freq_high_popu, index_col=0)
# all_times = np.load(save_file_all_times)
